# **Task 1**

In [14]:
from pyspark.sql import *
import pandas as pd

spark = SparkSession.builder.getOrCreate()
df = spark.read.text("ratings.txt")

AttributeError: 'DataFrame' object has no attribute 'a'